In [ ]:
pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
NUM_DAYS = 4000    # The number of days of historical data to retrieve
INTERVAL = '1d'     # Sample rate of historical data
symbol = 'AAPL'      # Symbol of the desired stock

In [ ]:
start = (datetime.date(2021, 12, 31) - datetime.timedelta(NUM_DAYS))
end = datetime.date(2021, 12, 31)
df = yf.download(symbol, start = start, end = end, interval = INTERVAL)
df.rename(columns = { 'Close' : 'close', 'Open' : 'open', 'High' : 'high', 'Low' : 'low', 'Volume' : 'volume'}, inplace = True)
df = df.iloc[:,3:4]
train_dataset = df.values

print(df)
print(df.shape)
print(train_dataset.shape)
# df['close'].plot()
tmp = df[-60:]
plt.plot(tmp)


In [ ]:
TEST_DAYS = 180
test_start = (datetime.date.today() - datetime.timedelta(TEST_DAYS))
test_end = datetime.date.today()
test_df = yf.download(symbol, start = test_start, end = test_end, interval = INTERVAL)
test_df.rename(columns = { 'Close' : 'close', 'Open' : 'open', 'High' : 'high', 'Low' : 'low', 'Volume' : 'volume'}, inplace = True)
test_df = test_df.iloc[:,3:4]
test_dataset = test_df.values

print(test_df)
print(test_df.shape)
tmp1 = test_df[-60:]
plt.plot(tmp1)



In [ ]:
#Preprocessing the data
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
train_dataset_scaled = sc.fit_transform(train_dataset)
print(train_dataset_scaled)

In [ ]:
#Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, train_dataset.shape[0]):
    X_train.append(train_dataset_scaled[i-60:i, 0])
    y_train.append(train_dataset_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
#Reshaping the input datas structure
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
print(X_train.shape)

In [ ]:
#Importing the keras packages
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [ ]:
#Initialising the regressor
regressor = Sequential()

In [ ]:
#Adding the first LSTM layer and some dropout regularisation layer
regressor.add(LSTM(units = 150, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
#Layer 2
regressor.add(LSTM(units = 150, return_sequences = True))
regressor.add(Dropout(0.2))
#Layer 3
regressor.add(LSTM(units = 150, return_sequences = True))
regressor.add(Dropout(0.2))
#Layer 4
regressor.add(LSTM(units = 150, return_sequences = False))
regressor.add(Dropout(0.2))

In [ ]:
#Output layer
regressor.add(Dense(units = 1))

In [ ]:
#Compile the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
#Getting the total dataset
total_dataset = pd.concat((df['close'], test_df['close']), axis = 0)
inputs = total_dataset[len(total_dataset)-len(test_df)-60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
print(inputs.shape)
print(len(test_df))

In [ ]:
#Creating the data structure
X_test = []
for i in range(60,60+len(test_df)):
    X_test.append(inputs[i-60:i, 0])
    
X_test = np.array(X_test)
# print(X_test.shape)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
#Plotting
plt.plot(test_dataset, color = 'red', label = 'Real Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Stock Price')
plt.title('Google Stock Price Prediction')
plt.legend()
plt.show()